In [1]:
%matplotlib inline

# モデルのパラメータを最適化する

モデルとデータができたので、今度はモデルのパラメータをデータに最適化することで、モデルのトレーニング、バリデーション、テストを行います。
モデルの学習は反復プロセスです。各反復（*epoch*と呼ばれる）において、モデルは出力を推測し、その推測の誤差（*loss*）を計算し、（モジュールで見たように）パラメータに関する誤差の導関数を収集し、勾配降下法を用いてパラメータを**最適化**します。このプロセスのより詳細なウォークスルーについては、[backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8)のビデオをご覧ください。

## 前提となるコード

前のモジュールの**Datasets & DataLoaders**と**Build Model**のコードを読み込みます。

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork()

## ハイパーパラメーターの設定 

ハイパーパラメータは、モデルの最適化プロセスをコントロールするための調整可能なパラメータです。
ハイパーパラメータの値が異なると、モデルの学習や収束率に影響します。
([続きを読む](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)ハイパーパラメータのチューニングについて)

トレーニング用に以下のハイパーパラメータを定義しています。

- **Number of Epochs** - データセットを反復処理する回数
- **Batch Size** - 各エポックでモデルが見るデータサンプルの数。
- **Learning Rate** - 各バッチ/エポックでモデルのパラメータをどの程度更新するか。小さな値では学習速度が遅く、大きな値では学習中に予測できない動作をする可能性があります。

In [3]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

## 最適化ループの追加

ハイパーパラメータを設定したら、最適化ループを使ってモデルのトレーニングと最適化を行います。
最適化ループの各繰り返しを **エポック** と呼びます。

各エポックは主に2つの部分で構成されています。

- **Train Loop** - トレーニングデータセットを繰り返し、最適なパラメータに収束させる。
- **Validation/Test Loop** - テストデータセットを繰り返し処理し、モデルの性能が向上しているかどうかを確認します。

トレーニングループで使われる概念を簡単に理解しておきましょう。
一足先に最適化ループの `full-impl-label` をご覧ください。

### 損失関数の追加

学習データを提示しても、学習していないネットワークは正しい答えを出さない可能性があります。
**損失関数**は、得られた結果の目標値に対する非類似度を測定します。
これが学習時に最小化したい損失関数です。損失を計算するために、与えられた入力を用いて予測を行います。
損失を計算するために、与えられたデータサンプルの入力を用いて予測を行い、それを真のデータラベル値と比較します。

一般的な損失関数には、回帰タスクのための[nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss) (Mean Square Error)や
分類のための[nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)（負の対数尤度）があります。
[nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss) は `nn.LogSoftmax` と `nn.NLLLoss` を組み合わせたものです。

モデルの出力logitsを`nn.CrossEntropyLoss`に渡し、logitsを正規化して予測誤差を計算します。

In [4]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

### 最適化パス

最適化とは、各学習ステップでモデルの誤差を減らすためにモデルのパラメータを調整するプロセスです。**最適化アルゴリズム**は、このプロセスの実行方法を定義します（この例では、確率的勾配降下法を使用します）。
すべての最適化ロジックは ``optimizer`` オブジェクトにカプセル化されています。ここでは、SGDオプティマイザを使用しています。
また、PADAMやRMSPropなど多くの[異なるオプティマイザ](https://pytorch.org/docs/stable/optim.html)があり、様々な種類のモデルやデータに適したオプティマイザーがPyTorchにはあります。

学習が必要なモデルのパラメータを登録し、学習率ハイパーパラメータを渡すことでオプティマイザを初期化します。

In [6]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

トレーニングループの中では3つのステップで最適化が行われます。

* モデルパラメータの勾配をリセットするために、`optimizer.zero_grad()`を呼び出します。勾配はデフォルトで加算されますが、二重にカウントされるのを防ぐために各反復で明示的にゼロにします。
* 予測損失を `loss.backwards()` の呼び出しで逆伝播させます。PyTorchは各パラメータに対する損失の勾配を取得します。
* 勾配が得られたら、``optimizer.step()``を呼び出し、バックワードパスで収集した勾配によってパラメータを調整します。

## 完全な実装

最適化されたコードをループする`train_loop`と、テストデータに対してモデルの性能を評価する`test_loop`を定義します。
テストデータに対してモデルのパフォーマンスを評価します。

In [7]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    test_loss, correct = 0, 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()

    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

損失関数とオプティマイザを初期化し，それを`train_loop`と`test_loop`に渡します．
モデルの性能が向上していることを確認するために、自由にエポック数を増やしてみてください。

In [8]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.296848  [    0/60000]
loss: 2.289259  [ 6400/60000]
loss: 2.281900  [12800/60000]
loss: 2.282959  [19200/60000]
loss: 2.252857  [25600/60000]
loss: 2.234976  [32000/60000]
loss: 2.251398  [38400/60000]
loss: 2.221005  [44800/60000]
loss: 2.214228  [51200/60000]
loss: 2.191263  [57600/60000]
Test Error: 
 Accuracy: 38.0%, Avg loss: 0.034320 

Epoch 2
-------------------------------
loss: 2.189986  [    0/60000]
loss: 2.181706  [ 6400/60000]
loss: 2.143917  [12800/60000]
loss: 2.168897  [19200/60000]
loss: 2.101498  [25600/60000]
loss: 2.064258  [32000/60000]
loss: 2.102571  [38400/60000]
loss: 2.035742  [44800/60000]
loss: 2.038806  [51200/60000]
loss: 1.991416  [57600/60000]
Test Error: 
 Accuracy: 52.9%, Avg loss: 0.031120 

Epoch 3
-------------------------------
loss: 1.999564  [    0/60000]
loss: 1.978428  [ 6400/60000]
loss: 1.889222  [12800/60000]
loss: 1.946574  [19200/60000]
loss: 1.816259  [25600/60000]
loss: 1.774364  [32000/600

モデルが最初はあまり良くないことに気付いたかもしれません（それでいいのです！）。もっと多くの `epochs` でループを実行するか、`learning_rate` をより大きな数値に調整してみてください。また、私たちが選んだモデル構成が、この種の問題に最適なものではないかもしれません（そうではありません）。この後の講座では、視覚問題に有効なモデルの形状について、より深く掘り下げていきます。

# 自分の知識を確認する

1. モデルの最適化における損失関数の目的は何ですか？

- 損失関数は、得られた結果と目標値の非類似性の度合いを測定します。
  > 不正解
- 損失関数の勾配は、学習時にオプティマイザが適切なパラメータを調整するのに役立ちます。
  > 不正解
- 損失関数は，学習時に最小化されるものである
  > 不正解
- 上記のすべてが当てはまる
  > 正解